In [1]:
from lerLaudo import lerLaudo
from LerSimuladorExcel import lerSimulador
from lerContrato import lerContrato

In [2]:
contrato = r"C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Contratos\HE_Contrato_Agostinho_Assinatura Digital_VFinal.pdf"
laudo = r'C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Laudos\5. Laudo - Agostinho.pdf'
simulador = r'C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Simuladores\HE Simulador Agostinho.xlsm'
import warnings
#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')


In [4]:
dadosContrato = lerContrato(contrato)
dadosContrato

{'valorTotal': '185236.16',
 'tabela': 'SAC',
 'registro': '3042.67',
 'Taxa ao Mes': 0.1162,
 'valorLiquido': '169200.00',
 'prazoMes': '240',
 'valorPrimeiraParcelaComEncargos': '2754.90',
 'valorImóvel': '376000.00',
 'prazoContrato': '234',
 'ultimaParcela': '30/06/2042',
 'dataContrato': '06/06/2022',
 'valorPrimeiraParcela': '2662.09',
 'primeiraParcela': '30/01/2023',
 'indice': 'IPCA',
 'Matrícula': '141.951',
 'Cartório': '02 de Registro Geral do 3º Oficial de Registro de Imóveis da Comarca de São Paulo/SP',
 'CCI': '0001013447/AFD',
 'Titular': 'EVEDORES FIDUCIANTES NOME: AGOSTINHO FRANCISCO DE SOUZA ',
 'Participante1': 'AGOSTINHO FRANCISCO DE SOUZA',
 'Participação1': '100%'}

In [3]:
from tkinter import *
import tkinter as tk
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
from time import sleep
import time
import os 
import warnings

#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='tkinter')

#Definir Variáveis Globais
contrato = None
simulador = None
laudo = None
textoContrato = None
botaoUm = None

ws = Tk()
ws.title('Validação Contrato x Simulador x Laudo')
ws.geometry('1900x1200') 


#Função Selecionar PDF
def open_file_pdf(entryBox):
    file_path = askopenfile(mode='r', filetypes=[('ALL', 'pdf')])
    entryBox.delete(0, 'end')
    if file_path is not None:
        pass
    entryBox.insert(END, file_path.name) # add this

#Função Selecionar Excel (xlsm)
def open_file_excel():
    textBoxSimulador.delete(0, 'end')
    file_path = askopenfile(mode='r', filetypes=[('ALL','*xlsm')])
    if file_path is not None:
        pass
    textBoxSimulador.insert(END, file_path.name) # add this

#Função para verificar se o path é válido
def verificarArquivo(arquivo):
    try:
        with open(arquivo, 'r') as f:
            print('Aquivo Existe')
    except IOError:
        print('Arquivo não encontrado!')

#Função para definir paths    
def savePaths():
    global contrato 
    global simulador
    global laudo

    contrato = textBoxContrato.get()
    laudo = textBoxLaudo.get()
    simulador = textBoxSimulador.get()
    if contrato != "" and laudo != "" and simulador != "":
        return 0
    else:
        return 1

#Ler o Contrato
def infosContrato(pathContrato):
    global textBoxContrato
    textoContrato = lerContrato(pathContrato)

    #Separar Key-Value
    valorBrutoC = str(textoContrato['valorTotal']).replace(".",",")
    valorLiquidoC = str(textoContrato['valorLiquido']).replace(".",",")
    valorAmortizacaoC = str(textoContrato['tabela'])
    valorIndiceC = str(textoContrato['indice'])
    prazoC = str(textoContrato['prazoMes'])
    parcelasContratoC = str(textoContrato['prazoContrato'])
    dataDesembolcoC = str(textoContrato['dataContrato'])
    dataPriParC = str(textoContrato['primeiraParcela'])
    dataUltimaParC = str(textoContrato['ultimaParcela'])
    valorPriParC = str(textoContrato['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosC = str(textoContrato['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoC = str(textoContrato['valorImóvel']).replace(".",",")
    matriculaImovelC = str(textoContrato['Matrícula'])
    cartorioImovelC = str(textoContrato['Cartório'])

    #Clear Box
    tbCValorBruto.delete(0, 'end')
    tbCValorLiquido.delete(0, 'end')
    tbCAmortizacao.delete(0, 'end')
    tbCIndice.delete(0, 'end')
    tbCPrazo.delete(0, 'end')
    tbCParcelas.delete(0, 'end')
    tbCDataDesem.delete(0, 'end')
    tbCDataPriParcela.delete(0, 'end')
    tbCDataUltimaParcela.delete(0, 'end')
    tbCValorPriPar.delete(0, 'end')
    tbCValorPriEncargosPar.delete(0, 'end')
    tbCValorAvaliacao.delete(0, 'end')
    tbCMatricula.delete(0, 'end')
    tbCCartorio.delete(0, 'end')


    #Inserir dados do Contrato
    tbCValorBruto.insert(END, valorBrutoC)
    tbCValorLiquido.insert(END, valorLiquidoC)
    tbCAmortizacao.insert(END, valorAmortizacaoC)
    tbCIndice.insert(END, valorIndiceC)
    tbCPrazo.insert(END, prazoC)
    tbCParcelas.insert(END, parcelasContratoC)
    tbCDataDesem.insert(END, dataDesembolcoC)
    tbCDataPriParcela.insert(END, dataPriParC)
    tbCDataUltimaParcela.insert(END, dataUltimaParC)
    tbCValorPriPar.insert(END, valorPriParC)
    tbCValorPriEncargosPar.insert(END, valorPriParEncargosC)
    tbCValorAvaliacao.insert(END, valorAvaliacaoC)
    tbCMatricula.insert(END, matriculaImovelC)
    tbCCartorio.insert(END, cartorioImovelC)
    
    return textoContrato

#Ler Simulador
def infosSimulador(pathSimulador):
    textoSimulador = lerSimulador(pathSimulador)
    
    #Separar Key-Value
    valorBrutoS = str(round(textoSimulador['valorTotal'],2)).replace(".",",")
    valorLiquidoS = str(textoSimulador['valorLiquido']).replace(".",",")
    valorAmortizacaoS = str(textoSimulador['tabela'])
    valorIndiceS = str(textoSimulador['indice'])
    prazoS = str(textoSimulador['prazoMes'])
    parcelasContratoS = str(textoSimulador['prazoContrato'])
    dataDesembolcoS = str(textoSimulador['dataContrato'])
    dataPriParS = str(textoSimulador['primeiraParcela'])
    dataUltimaParS = str(textoSimulador['ultimaParcela'])
    valorPriParS = str(textoSimulador['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosS = str(textoSimulador['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoS = str(textoSimulador['valorImóvel']).replace(".",",")

    #Clear Box
    tbSValorBruto.delete(0, 'end')
    tbSValorLiquido.delete(0, 'end')
    tbSAmortizacao.delete(0, 'end')
    tbSIndice.delete(0, 'end')
    tbSPrazo.delete(0, 'end')
    tbSParcelas.delete(0, 'end')
    tbSDataDesem.delete(0, 'end')
    tbSDataPriParcela.delete(0, 'end')
    tbSDataUltimaParcela.delete(0, 'end')
    tbSValorPriPar.delete(0, 'end')
    tbSValorPriEncargosPar.delete(0, 'end')
    tbSValorAvaliacao.delete(0, 'end')

    #Inserir dados do Contrato
    tbSValorBruto.insert(END, valorBrutoS)
    tbSValorLiquido.insert(END, valorLiquidoS)
    tbSAmortizacao.insert(END, valorAmortizacaoS)
    tbSIndice.insert(END, valorIndiceS)
    tbSPrazo.insert(END, prazoS)
    tbSParcelas.insert(END, parcelasContratoS)
    tbSDataDesem.insert(END, dataDesembolcoS)
    tbSDataPriParcela.insert(END, dataPriParS)
    tbSDataUltimaParcela.insert(END, dataUltimaParS)
    tbSValorPriPar.insert(END, valorPriParS)
    tbSValorPriEncargosPar.insert(END, valorPriParEncargosS)
    tbSValorAvaliacao.insert(END, valorAvaliacaoS)

#Ler o Laudo
def infosLaudo(pathLaudo):
    textoLaudo = lerLaudo(pathLaudo)

    #Separar Key-Value
    valorAvaliacaoL = str(textoLaudo['Valor de Mercado: '])
    matriculaImovelL = str(textoLaudo['Matrícula: '])
    cartorioImovelL = str(textoLaudo['Cartório'])

    #Clear Box
    tbLValorAvaliacao.delete(0, 'end')
    tbLMatricula.delete(0, 'end')
    tbLCartorio.delete(0, 'end')

    #Inserir dados do Contrato
    tbLValorAvaliacao.insert(END, valorAvaliacaoL)
    tbLMatricula.insert(END, matriculaImovelL)
    tbLCartorio.insert(END, cartorioImovelL)

#Função Validar Itens
def validarEntry():
    
    #Var Globais Contrato
    global tbCValorBruto
    global tbCValorLiquido
    global tbCAmortizacao
    global tbCIndice
    global tbCPrazo
    global tbCParcelas
    global tbCDataDesem
    global tbCDataPriParcela
    global tbCDataUltimaParcela
    global tbCValorPriPar
    global tbCValorPriEncargosPar
    global tbCValorAvaliacao
    global tbCMatricula
    global tbCCartorio

    #Var Globais Simulador
    global tbSValorBruto
    global tbSValorLiquido
    global tbSAmortizacao
    global tbSIndice
    global tbSPrazo
    global tbSParcelas
    global tbSDataDesem
    global tbSDataPriParcela
    global tbSDataUltimaParcela
    global tbSValorPriPar
    global tbSValorPriEncargosPar
    global tbSValorAvaliacao

    #Var Globais Laudo
    global tbLValorAvaliacao
    global tbLMatricula
    global tbLCartorio

    #Definção de Lista de Contrato
    listaContrato = [tbCValorBruto, tbCValorLiquido, tbCAmortizacao, tbCIndice, tbCPrazo, tbCParcelas, tbCDataDesem, tbCDataPriParcela,
                    tbCDataUltimaParcela, tbCValorPriPar, tbCValorPriEncargosPar, tbCValorAvaliacao, tbCMatricula, tbCCartorio
                    ]

    #Definção de Lista do Simulador
    listaSimulador = [tbSValorBruto, tbSValorLiquido, tbSAmortizacao, tbSIndice, tbSPrazo, tbSParcelas, tbSDataDesem, tbSDataPriParcela,
                        tbSDataUltimaParcela, tbSValorPriPar, tbSValorPriEncargosPar, tbSValorAvaliacao, tbLMatricula, tbLCartorio
                    ]
    
    tamanhoLista = len(listaContrato)

    for i in range(tamanhoLista):
        num = i + 1
        varContrato = listaContrato[i].get() 
        varSimulador = listaSimulador[i].get()
        
        #Corrigir Espaço Indesejado
        if i<13:
            varContrato = varContrato.replace(" ", "")
            varSimulador = varSimulador.replace(" ", "")
        
        #Transformar Valores Númericos
        if i == 0 or i == 1 or i == 9 or i == 10 or i == 11:
            varContrato = round(float(varContrato.replace(",",".")),2)
            varSimulador = round(float(varSimulador.replace(",",".")),2)

        if i == 2 or i == 3:
            varContrato = varContrato.lower()
            varSimulador = varSimulador.lower()

        #Comparação de Valores
        if varContrato == varSimulador:
            botaoUm = "VERDADEIRO"
            color = 'green'
            fontColor = 'white'
            
        else:
            botaoUm = "FALSO"
            color = 'red'
            fontColor = 'white'
    
        # #Criar Botão de Validação
        validValorBruto = tk.Button(
             ws,
             text = botaoUm,
             bg=color,
             fg=fontColor
             )
        validValorBruto.grid(row=8+num, column=2)

#Função UploadFiles
def uploadFiles():
    global contrato
    pb1 = Progressbar(
        ws, 
        orient=HORIZONTAL, 
        length=300, 
        mode='determinate'
        )
    pb1.grid(row=4, columnspan=3, pady=20)
    for i in range(5):
        ws.update_idletasks()
        pb1['value'] += 20
        time.sleep(1)
    pb1.destroy()
    msg = savePaths()
    if msg == 0:
        textoUpload = "Todos os arquivos foram anexados corretamente. Siga com a validação dos dados apresentados abaixo."
        color = 'green'
    elif msg ==1:
        textoUpload = "Favor selecionar todos os arquivos antes de seguir para os próximos passos."
        color = 'red'
    Label(ws, text=textoUpload, foreground=color).grid(row=4, columnspan=3, pady=10)
    infosContrato(contrato)
    infosSimulador(simulador)
    infosLaudo(laudo)



#Botões relacionados ao Contrato
textBoxContrato = Entry(
    ws, 
    width=30
    )
textBoxContrato.grid(row=0, column=2)    
botaoContrato = Label(
    ws, 
    text='Selecione o contrato em PDF'
    )
botaoContrato.grid(row=0, column=0, padx=10)
anexarContrato = Button(
    ws, 
    text ='Selecione o Contrato', 
    command = lambda:open_file_pdf(textBoxContrato)
    ) 
anexarContrato.grid(row=0, column=1)

#Botões Simulador
textBoxSimulador = Entry(
    ws,
    width=30,
    
    )
textBoxSimulador.grid(row=1, column=2)    
botaoSimulador = Label(
    ws, 
    text='Selecione o simulador em Excel'
    )
botaoSimulador.grid(row=1, column=0, padx=10)
anexarSimulador = Button(
    ws, 
    text ='Selecione o Simulador', 
    command = lambda:open_file_excel()
    ) 
anexarSimulador.grid(row=1, column=1)

#Botões Laudo
textBoxLaudo = Entry(
    ws,
    width=30
    )
textBoxLaudo.grid(row=2, column=2)
botaoLaudo = Label(
    ws, 
    text='Selecione o Laudo em PDF'
    )
botaoLaudo.grid(row=2, column=0, padx=10)
anexarLaudo = Button(
    ws, 
    text ='Selecione o Laudo', 
    command = lambda:open_file_pdf(textBoxLaudo)
    ) 
anexarLaudo.grid(row=2, column=1)

#Botão Importar Valores
upld = tk.Button(
    ws, 
    text='Importar Valores', 
    command=uploadFiles
    )
upld.grid(row=3, columnspan=1, pady=10)

#Botão Validar
conf = Button(
    ws, 
    text='Validar Valores', 
    command=validarEntry
    )
conf.grid(row=3, columnspan=5, pady=10)

#Menus
dadosContrato = Label(
    ws, 
    text='Dados do Contrato',
    font=40
    )
dadosContrato.grid(row=6, column=0)
dadosSimulador = Label(
    ws, 
    text='Dados do Simulador',
    font=40,
    )
dadosSimulador.grid(row=6, column=4, padx=30)
dadosLaudo = Label(
    ws, 
    text='Dados do Laudo',
    font=40,
    )
dadosLaudo.grid(row=6, column=7, padx=150)

#Saída de Dados Contratos
try:
    #Valor Bruto
    try:
        botaovalorBrutoC = Label(
            ws, 
            text='Valor Bruto'
            )
        botaovalorBrutoC.grid(row=9, column=0)
        tbCValorBruto = Entry(
            ws,
            width=20
            )
        tbCValorBruto.grid(row=9, column=1)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoC = Label(
            ws, 
            text='Valor Liquido'
            )
        botaovalorLiquidoC.grid(row=10, column=0)
        tbCValorLiquido = Entry(
            ws,
            width=20
            )
        tbCValorLiquido.grid(row=10, column=1)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoC = Label(
            ws, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoC.grid(row=11, column=0)
        tbCAmortizacao = Entry(
            ws,
            width=20
            )
        tbCAmortizacao.grid(row=11, column=1)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceC = Label(
            ws, 
            text='Indice de Correção'
            )
        botaoIndiceC.grid(row=12, column=0)
        tbCIndice = Entry(
            ws,
            width=20
            )
        tbCIndice.grid(row=12, column=1)
    except:
        pass

    #Prazo
    try:
        botaoPrazoC = Label(
            ws, 
            text='Prazo da Operação'
            )
        botaoPrazoC.grid(row=13, column=0)
        tbCPrazo = Entry(
            ws,
            width=20
            )
        tbCPrazo.grid(row=13, column=1)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasC = Label(
            ws, 
            text='Número de parcelas'
            )
        botaoParcelasC.grid(row=14, column=0)
        tbCParcelas = Entry(
            ws,
            width=20
            )
        tbCParcelas.grid(row=14, column=1)
    except:
        pass

    #Data Desembolço
    try:
        botaoDataDesemC = Label(
            ws, 
            text='Data de Desembolço'
            )
        botaoDataDesemC.grid(row=15, column=0)
        tbCDataDesem = Entry(
            ws,
            width=20
            )
        tbCDataDesem.grid(row=15, column=1)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaC = Label(
            ws, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaC.grid(row=16, column=0)
        tbCDataPriParcela = Entry(
            ws,
            width=20
            )
        tbCDataPriParcela.grid(row=16, column=1)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaC = Label(
            ws, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaC.grid(row=17, column=0)
        tbCDataUltimaParcela = Entry(
            ws,
            width=20
            )
        tbCDataUltimaParcela.grid(row=17, column=1)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParC = Label(
            ws, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParC.grid(row=18, column=0)
        tbCValorPriPar = Entry(
            ws,
            width=20
            )
        tbCValorPriPar.grid(row=18, column=1)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosC = Label(
            ws, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosC.grid(row=19, column=0)
        tbCValorPriEncargosPar = Entry(
            ws,
            width=20
            )
        tbCValorPriEncargosPar.grid(row=19, column=1)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoC = Label(
            ws, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoC.grid(row=20, column=0)
        tbCValorAvaliacao = Entry(
            ws,
            width=20
            )
        tbCValorAvaliacao.grid(row=20, column=1)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaC = Label(
            ws, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaC.grid(row=21, column=0)
        tbCMatricula = Entry(
            ws,
            width=20
            )
        tbCMatricula.grid(row=21, column=1)
    except:
        pass

    #Cartório
    try:
        botaoCartorioC = Label(
            ws, 
            text='Cartório'
            )
        botaoCartorioC.grid(row=22, column=0)
        tbCCartorio = Entry(
            ws,
            width=20
            )
        tbCCartorio.grid(row=22, column=1)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")


#Saída de Dados Simulador
try:
    #Valor Bruto
    try:
        botaovalorBrutoS = Label(
            ws, 
            text='Valor Bruto'
            )
        botaovalorBrutoS.grid(row=9, column=4)
        tbSValorBruto = Entry(
            ws,
            width=20
            )
        tbSValorBruto.grid(row=9, column=5)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoS = Label(
            ws, 
            text='Valor Liquido'
            )
        botaovalorLiquidoS.grid(row=10, column=4)
        tbSValorLiquido = Entry(
            ws,
            width=20
            )
        tbSValorLiquido.grid(row=10, column=5)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoS = Label(
            ws, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoS.grid(row=11, column=4)
        tbSAmortizacao = Entry(
            ws,
            width=20
            )
        tbSAmortizacao.grid(row=11, column=5)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceS = Label(
            ws, 
            text='Indice de Correção'
            )
        botaoIndiceS.grid(row=12, column=4)
        tbSIndice = Entry(
            ws,
            width=20
            )
        tbSIndice.grid(row=12, column=5)
    except:
        pass

    #Prazo
    try:
        botaoPrazoS = Label(
            ws, 
            text='Prazo da Operação'
            )
        botaoPrazoS.grid(row=13, column=4)
        tbSPrazo = Entry(
            ws,
            width=20
            )
        tbSPrazo.grid(row=13, column=5)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasS = Label(
            ws, 
            text='Número de parcelas'
            )
        botaoParcelasS.grid(row=14, column=4)
        tbSParcelas = Entry(
            ws,
            width=20
            )
        tbSParcelas.grid(row=14, column=5)
    except:
        pass

    #Data Desembolço
    try:
        botaoDataDesemS = Label(
            ws, 
            text='Data de Desembolço'
            )
        botaoDataDesemS.grid(row=15, column=4)
        tbSDataDesem = Entry(
            ws,
            width=20
            )
        tbSDataDesem.grid(row=15, column=5)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaS = Label(
            ws, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaS.grid(row=16, column=4)
        tbSDataPriParcela = Entry(
            ws,
            width=20
            )
        tbSDataPriParcela.grid(row=16, column=5)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaS = Label(
            ws, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaS.grid(row=17, column=4)
        tbSDataUltimaParcela = Entry(
            ws,
            width=20
            )
        tbSDataUltimaParcela.grid(row=17, column=5)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParS = Label(
            ws, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParS.grid(row=18, column=4)
        tbSValorPriPar = Entry(
            ws,
            width=20
            )
        tbSValorPriPar.grid(row=18, column=5)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosS = Label(
            ws, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosS.grid(row=19, column=4)
        tbSValorPriEncargosPar = Entry(
            ws,
            width=20
            )
        tbSValorPriEncargosPar.grid(row=19, column=5)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoS = Label(
            ws, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoS.grid(row=20, column=4)
        tbSValorAvaliacao = Entry(
            ws,
            width=20
            )
        tbSValorAvaliacao.grid(row=20, column=5)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

#Saída de Dados Laudo
try:
    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoL = Label(
            ws, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoL.grid(row=20, column=7)
        tbLValorAvaliacao = Entry(
            ws,
            width=20
            )
        tbLValorAvaliacao.grid(row=20, column=8)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaL = Label(
            ws, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaL.grid(row=21, column=7)
        tbLMatricula = Entry(
            ws,
            width=20
            )
        tbLMatricula.grid(row=21, column=8)
    except:
        pass

    #Cartório
    try:
        botaoCartorioL = Label(
            ws, 
            text='Cartório'
            )
        botaoCartorioL.grid(row=22, column=7)
        tbLCartorio = Entry(
            ws,
            width=20
            )
        tbLCartorio.grid(row=22, column=8)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

ws.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Matheus\AppData\Local\Temp\ipykernel_21448\1556112564.py", line 298, in uploadFiles
    infosLaudo(laudo)
  File "C:\Users\Matheus\AppData\Local\Temp\ipykernel_21448\1556112564.py", line 167, in infosLaudo
    textoLaudo = lerLaudo(pathLaudo)
  File "c:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Scripts\lerLaudo.py", line 7, in lerLaudo
    extTexto = lerPDF(pathDocument,4)
  File "c:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Scripts\lerPDFLaudo.py", line 24, in lerPDF
    pageObj = read_pdf.getPage(i)
  File "c:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\site-packages\PyPDF2\_reader.py", line 423, in getPage
    return self._get_page(pageNumber)
  File "c:\Users\Matheus\Ap

'C:/Users/Matheus/Documents/Git/Pontte/automacaoRegistroAztronic/automacaoRegistroCCI/Laudos/5. Laudo - Agostinho.pdf'